In [54]:
from typing import Dict,Text
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import duckdb
con = duckdb.connect('database.db')

#PREPARE DATA

podcast_ratings_query = con.sql("select author_id as user_id, podcasts.title as podcast_title,rating from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_titles_query = con.sql("select podcasts.title as podcast_title from reviews "+ 
        "join categories using (podcast_id) " +
        "join podcasts using (podcast_id) where average_rating >= 0").to_df()

podcast_ratings = podcast_ratings_query.to_dict(orient='records')
podcast_titles = podcast_titles_query.to_dict(orient='records')
podcast_ratings_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_ratings).to_dict(orient="list"))
podcast_titles_tf = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(podcast_titles).to_dict(orient="list"))

ratings_tf = podcast_ratings_tf.map(lambda x: {
    "podcast_title": x["podcast_title"],
    "user_id": x["user_id"],
    "user_rating": x["rating"]
})
titles_tf = podcast_titles_tf.map(lambda x: x["podcast_title"])

tf.random.set_seed(42)
shuffled = ratings_tf.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train_len = int(len(shuffled) * (2/3))
test_len = int(len(shuffled) - train_len)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)

podcast_titles = titles_tf.batch(1_000)
user_ids = ratings_tf.batch(1_000_000).map(lambda x: x["user_id"])

unique_podcast_titles = np.unique(np.concatenate(list(podcast_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [55]:
#TRAIN MODEL

class PodcastlensModel(tfrs.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32
    self.user_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_user_ids, mask_token=None),
      # We add an additional embedding to account for unknown tokens.
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    self.podcast_model = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_podcast_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_podcast_titles) + 1, embedding_dimension)
    ])
    metrics = tfrs.metrics.FactorizedTopK(
      candidates=titles_tf.batch(128).map(self.podcast_model)
    )

    self.task = tfrs.tasks.Retrieval(
      metrics=metrics
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the podcast features and pass them into the podcast model,
    # getting embeddings back.
    positive_podcast_embeddings = self.podcast_model(features["podcast_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_podcast_embeddings)

model = PodcastlensModel()
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.1))
cached_train = train.shuffle(100_000).batch(250).cache()
cached_test = test.batch(100).cache()
model.fit(cached_train, epochs=25)

# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.Streaming(model.user_model,k=25)
# recommends podcasts out of the entire podcasts dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((titles_tf.batch(100), titles_tf.batch(100).map(model.podcast_model)))
)

Epoch 1/25
3/3 [==============================] - 1s 65ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0019 - factorized_top_k/top_5_categorical_accuracy: 0.0058 - factorized_top_k/top_10_categorical_accuracy: 0.0117 - factorized_top_k/top_50_categorical_accuracy: 0.0585 - factorized_top_k/top_100_categorical_accuracy: 0.1053 - loss: 702.9679 - regularization_loss: 0.0000e+00 - total_loss: 702.9679             
Epoch 2/25
3/3 [==============================] - 0s 62ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0292 - factorized_top_k/top_10_categorical_accuracy: 0.0760 - factorized_top_k/top_50_categorical_accuracy: 0.2651 - factorized_top_k/top_100_categorical_accuracy: 0.4522 - loss: 627.0885 - regularization_loss: 0.0000e+00 - total_loss: 627.0885
Epoch 3/25
3/3 [==============================] - 0s 61ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_acc

In [56]:
#PULL METRICS FOR MODEL ON TEST DATA 
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 1s 87ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5292 - factorized_top_k/top_5_categorical_accuracy: 0.5837 - factorized_top_k/top_10_categorical_accuracy: 0.5992 - factorized_top_k/top_50_categorical_accuracy: 0.6732 - factorized_top_k/top_100_categorical_accuracy: 0.7198 - loss: 224.4615 - regularization_loss: 0.0000e+00 - total_loss: 224.4615


{'factorized_top_k/top_1_categorical_accuracy': 0.5291828513145447,
 'factorized_top_k/top_5_categorical_accuracy': 0.5836575627326965,
 'factorized_top_k/top_10_categorical_accuracy': 0.5992217659950256,
 'factorized_top_k/top_50_categorical_accuracy': 0.6731517314910889,
 'factorized_top_k/top_100_categorical_accuracy': 0.7198443412780762,
 'loss': 146.1494903564453,
 'regularization_loss': 0,
 'total_loss': 146.1494903564453}

In [57]:
#PULL METRICS FOR MODEL ON TRAIN DATA 
model.evaluate(cached_train, return_dict=True)

3/3 [==============================] - 0s 90ms/step - factorized_top_k/top_1_categorical_accuracy: 0.6101 - factorized_top_k/top_5_categorical_accuracy: 0.6784 - factorized_top_k/top_10_categorical_accuracy: 0.6920 - factorized_top_k/top_50_categorical_accuracy: 0.7661 - factorized_top_k/top_100_categorical_accuracy: 0.8402 - loss: 401.6993 - regularization_loss: 0.0000e+00 - total_loss: 401.6993


{'factorized_top_k/top_1_categorical_accuracy': 0.6101364493370056,
 'factorized_top_k/top_5_categorical_accuracy': 0.6783625483512878,
 'factorized_top_k/top_10_categorical_accuracy': 0.6920077800750732,
 'factorized_top_k/top_50_categorical_accuracy': 0.7660818696022034,
 'factorized_top_k/top_100_categorical_accuracy': 0.8401559591293335,
 'loss': 10.227550506591797,
 'regularization_loss': 0,
 'total_loss': 10.227550506591797}

In [58]:
#RUN PREDICTIONS ON TEST DATA 
num = 0 
threshold = 1
for element in test:
    user_id = element["user_id"].numpy().decode()
    _, titles = index(tf.constant([user_id]))
    unique_preds = np.unique(titles.numpy())
    unique_preds = [el.decode('UTF-8') for el in unique_preds]
    if len(unique_preds) > threshold:
        print("Recommendations for user {}: {}".format(user_id,unique_preds))
        print("")
        num += 1
print("Number of users with more than {} recommendation: {}".format(threshold,num))

Recommendations for user d732db86069c70c: ['Maximiza Tu Negocio en Redes de Mercadeo', 'Talking Web Marketing', 'The Oatley Academy ArtCast']

Recommendations for user 98fd44d2cb0581e: ['Daily Solutions Podcast', 'Outside Voices Podcast', 'Sketched Out', 'Understanding Human Behavior - Video']

Recommendations for user 5eac2a0f1457a77: ['Hear It Now (retired)', 'Last Week at the Movies Podcast', 'Little Realms | A DnD Actual Play Podcast', 'Twin Talk with the King Twins']

Recommendations for user d9920e217cc1d6c: ['Hear It Now (retired)', 'Last Week at the Movies Podcast', 'New Patient Group™ (Formally known as the Doctor Diamond Club Podcast)', 'Talking Web Marketing']

Recommendations for user 9143fc8a95f5f9f: ['Daily Solutions Podcast', 'Maximiza Tu Negocio en Redes de Mercadeo', 'Nahh B! Podcast MMA / UFC And Boxing Event Preview & Reviews', 'Sketched Out', 'Understanding Human Behavior - Video']

Recommendations for user d9920e217cc1d6c: ['Hear It Now (retired)', 'Last Week at th